# **Homework 8 - Anomaly Detection**

If there are any questions, please contact mlta-2022spring-ta@googlegroups.com

Slide:    [Link]()　Kaggle: [Link](https://www.kaggle.com/c/ml2022spring-hw8)

# Set up the environment


## Package installation

In [1]:
# Training progress bar
!pip install -q qqdm

## Downloading data

In [2]:
!wget https://github.com/MachineLearningHW/HW8_Dataset/releases/download/v1.0.0/data.zip

In [3]:
!nvidia-smi

Sat May  7 14:51:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:1B:00.0 Off |                  N/A |
| 30%   35C    P8     3W / 225W |  10090MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  On   | 00000000:1C:00.0 Off |                  N/A |
| 29%   

In [4]:

# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "6"


In [5]:
 !unzip data.zip

In [6]:
# from shutil import copyfile
# copyfile(src = "../input/hw8-data/testingset.npy", dst = "./testingset.npy")
# copyfile(src = "../input/hw8-data/trainingset.npy", dst = "./trainingset.npy")


In [7]:
!pip install torchvision

# Import packages

In [8]:
!pip install pandas

In [9]:
import random
import numpy as np
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
from torch.optim import Adam, AdamW
from qqdm import qqdm, format_str
import pandas as pd

# Loading data

In [10]:

train = np.load('data/trainingset.npy', allow_pickle=True)
test = np.load('data/testingset.npy', allow_pickle=True)

print(train.shape)
print(test.shape)

(100000, 64, 64, 3)
(19636, 64, 64, 3)


## Random seed
Set the random seed to a certain value for reproducibility.

In [11]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(14132)

# Autoencoder

# Models & loss

In [36]:
class fcn_autoencoder(nn.Module):
    def __init__(self):
        super(fcn_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(64 * 64 * 3, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(), 
            nn.Linear(64, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        
        self.encoder1 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(), 
            nn.Linear(64, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        
        self.encoder2 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 2048),
            nn.ReLU(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        self.encoder4 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 300),
            nn.ReLU(),
            nn.Linear(300, 256),
            nn.ReLU(), 
            nn.Linear(256, 128), 
            nn.ReLU(), 
            nn.Linear(128, 64), 
            nn.ReLU(), 
            nn.Linear(64, 32), 
            nn.ReLU(), 
            nn.Linear(32, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        self.encoder5 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),   
            nn.Linear(64, 32),
            nn.ReLU(), 
            nn.Linear(32, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        
        self.encoder6 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(),
            nn.Linear(32, 12),
            nn.ReLU(),
            nn.Linear(12, 3)
        )
        self.encoder7 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 12),
            nn.ReLU(),
            nn.Linear(12, 3)
        )
            
        self.encoder8 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 184),
            nn.ReLU(),
            nn.Linear(184, 92),
            nn.ReLU(),
            nn.Linear(92, 48),
            nn.ReLU(),
            nn.Linear(48, 12),
            nn.ReLU(),
            nn.Linear(12, 3)
        )
        self.encoder9 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU(), 
            nn.Linear(32, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        
        self.encoder10 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(), 
            nn.Linear(32, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        
        
        self.encoder11 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 250),
            nn.ReLU(),
            nn.Linear(250, 150),
            nn.ReLU(),
            nn.Linear(150, 128),
            nn.ReLU(),
            nn.Linear(128, 84),
            nn.ReLU(),
            nn.Linear(84, 48),
            nn.ReLU(), 
            nn.Linear(48, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )
        
        self.encoder12 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 300),
            nn.ReLU(),
            nn.Linear(300, 200),
            nn.ReLU(),
            nn.Linear(200, 100),
            nn.ReLU(),
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(), 
            nn.Linear(32, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )


        self.encoder13 = nn.Sequential(
            nn.Linear(64 * 64 * 3, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 48),
            nn.ReLU(),
            nn.Linear(48, 12), 
            nn.ReLU(), 
            nn.Linear(12, 3)
        )

        self.decoder = nn.Sequential(
            nn.Linear(39, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 150),
            nn.ReLU(),
            nn.Linear(150, 252),
            nn.ReLU(),
            nn.Linear(252, 300),
            nn.ReLU(),
            nn.Linear(300, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 64 * 64 * 3), 
            nn.Tanh()
        )


    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.encoder1(x)
        x3 = self.encoder2(x)
        x4 = self.encoder4(x)
        x5 = self.encoder5(x)
        x6 = self.encoder6(x)
        x7 = self.encoder7(x)
        x8 = self.encoder8(x)
        x9 = self.encoder9(x)
        x10 = self.encoder10(x)
        x11 = self.encoder11(x)
        x12 = self.encoder12(x)
        x13 = self.encoder13(x)
        
        x14 = torch.cat((x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13),1)
        # x11 = torch.cat((x1,x4,x7,x8),1)
        x15 = self.decoder(x14)
        return x15


class conv_autoencoder(nn.Module):
    def __init__(self):
        super(conv_autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(12, 32, 4, stride=2, padding=1),        
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),  
            nn.ReLU(),
        )    
        self.encoder1 = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(12, 32, 4, stride=2, padding=1),        
            nn.ReLU(),
            nn.Conv2d(32, 75, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(75, 150, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(150, 300, 4, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(300, 512, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.encoder2 = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(12, 48, 4, stride=2, padding=1),        
            nn.ReLU(),
            nn.Conv2d(48, 96, 4, stride=2, padding=1),         
            nn.ReLU(),
            nn.Conv2d(96, 128, 4, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),  
            nn.ReLU(),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),
            nn.Tanh(),
        )



    def forward(self, x):
        x1 = self.encoder(x)
        # print(x1.shape)
        x2 = self.encoder1(x)
        # print(x2.shape)
        x3 = self.encoder2(x)
        # print(x3.shape)
        
        x4 = x1+x2+x3
        # print(x4.shape)
        x = self.decoder(x4)
        
        return x


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),    
            nn.ReLU(),
        )
        self.enc_out_1 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),  
            nn.ReLU(),
        )
        self.enc_out_2 = nn.Sequential(
            nn.Conv2d(24, 48, 4, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), 
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), 
            nn.Tanh(),
        )

    def encode(self, x):
        h1 = self.encoder(x)
        return self.enc_out_1(h1), self.enc_out_2(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


def loss_vae(recon_x, x, mu, logvar, criterion):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    mse = criterion(recon_x, x)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    return mse + KLD

# Dataset module

Module for obtaining and processing data. The transform function here normalizes image's pixels from [0, 255] to [-1.0, 1.0].


In [37]:
class CustomTensorDataset(TensorDataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors):
        self.tensors = tensors
        if tensors.shape[-1] == 3:
            self.tensors = tensors.permute(0, 3, 1, 2)
        
        self.transform = transforms.Compose([
          transforms.Lambda(lambda x: x.to(torch.float32)),
          transforms.Lambda(lambda x: 2. * x/255. - 1.),
        ])
        
    def __getitem__(self, index):
        x = self.tensors[index]
        
        if self.transform:
            # mapping images to [-1.0, 1.0]
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.tensors)

# Training

## Configuration


In [38]:
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR
# Training hyperparameters
num_epochs = 512
batch_size = 10000
learning_rate = 1e-3

# Build training dataloader
x = torch.from_numpy(train)
train_dataset = CustomTensorDataset(x)

train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
train_num = len(train_dataloader)

# Model
model_type = 'fcn'   # selecting a model type from {'cnn', 'fcn', 'vae', 'resnet'}
model_classes = {'fcn': fcn_autoencoder(), 'cnn': conv_autoencoder(), 'vae': VAE()}
model = model_classes[model_type].cuda()

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# scheduler = lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.1)
##step_size=args.step_size
 

## Training loop

In [39]:

best_loss = np.inf
model.train()

qqdm_train = qqdm(range(num_epochs), desc=format_str('bold', 'Description'))
for epoch in qqdm_train:
    tot_loss = list()
    for data in train_dataloader:

        # ===================loading=====================
        img = data.float().cuda()
        if model_type in ['fcn']:
            img = img.view(img.shape[0], -1)

        # ===================forward=====================
        output = model(img)
        if model_type in ['vae']:
            loss = loss_vae(output[0], img, output[1], output[2], criterion)
        else:
            loss = criterion(output, img)

        tot_loss.append(loss.item())
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         scheduler.step()
    # ===================save_best====================
    mean_loss = np.mean(tot_loss)
    if mean_loss < best_loss:
        best_loss = mean_loss
        torch.save(model, 'best_model_{}.pt'.format(model_type))
    # ===================log========================
    qqdm_train.set_infos({
        'epoch': f'{epoch + 1:.0f}/{num_epochs:.0f}',
        'loss': f'{mean_loss:.4f}',
    })
    # ===================save_last========================
    torch.save(model, 'last_model_{}.pt'.format(model_type))

 Iters    Elapsed Time      Speed                                                                                       
 0/512          -             -                                                                                         
Description   0.0% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 1/512  00:00:17<02:27:51  0.06it/s  1/512  0.3345                                                                      
Description   0.2% |                                                                                                   | Iters    Elapsed Time      Speed    epoch   loss                                                                       
 2/512  00:00:34<02:26:06  0.06it/s  2/512  0.2666                                                                      
Description   0.4% |              

# Inference
Model is loaded and generates its anomaly score predictions.

## Initialize
- dataloader
- model
- prediction file

In [40]:
eval_batch_size = 200

# build testing dataloader
data = torch.tensor(test, dtype=torch.float32)
test_dataset = CustomTensorDataset(data)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=eval_batch_size, num_workers=1)
eval_loss = nn.MSELoss(reduction='none')

# load trained model
checkpoint_path = f'last_model_{model_type}.pt'
model = torch.load(checkpoint_path)
model.eval()

# prediction file 
out_file = 'prediction.csv'

In [41]:
anomality = list()
with torch.no_grad():
  for i, data in enumerate(test_dataloader):
    img = data.float().cuda()
    if model_type in ['fcn']:
      img = img.view(img.shape[0], -1)
    output = model(img)
    if model_type in ['vae']:
      output = output[0]
    if model_type in ['fcn']:
        loss = eval_loss(output, img).sum(-1)
    else:
        loss = eval_loss(output, img).sum([1, 2, 3])
    anomality.append(loss)
anomality = torch.cat(anomality, axis=0)
anomality = torch.sqrt(anomality).reshape(len(test), 1).cpu().numpy()

df = pd.DataFrame(anomality, columns=['score'])
df.to_csv(out_file, index_label = 'ID')